## 任务3：基于 Python 快速构建带安全认证的 MCP 服务端与客户端，开发生产级工具链

在**任务1**和**任务2**中，我们已经了解了 MCP 协议的基础架构、Agent 运行模式，以及如何利用现有 MCP 服务实现动态工具调用与上下文同步。本任务将进一步深入，指导你**如何从零开始，使用 Python 构建一个具备安全认证的 MCP 服务端**，并将其与 AI Agent 结合，从而在 Cursor IDE 环境下开发出生产级的工具链。

自定义 MCP 服务端的核心价值在于，它允许你将任何**Python 逻辑、内部 API 或本地计算资源**封装为可供 AI Agent 调用的工具。通过引入**安全认证机制**，你可以确保只有获得授权的应用程序才能安全、可控地访问这些自定义能力。


在 任务1 和 任务2 中，我们了解了 MCP 协议的基础架构、Agent 运行模式，以及如何通过现有 MCP 服务进行动态工具调用和上下文同步。本任务将更进一步，指导你如何基于 Python 从零开始构建一个带安全认证的 MCP 服务端，并将其集成到 Cursor IDE 的 mcp.json 配置中，从而开发出生产级的 AI 工具链。

构建自定义 MCP 服务端的核心优势在于，你可以将任何 Python 逻辑、内部 API 或本地资源封装为 AI Agent 可调用的工具，并且通过安全认证机制，确保只有授权的应用程序才能访问这些能力。

### 1. 构建带安全认证的 Python MCP 服务端

本节将创建一个简单的 Python MCP 服务端，它提供一个受认证保护的“问候”功能，只会返回一个预设的问候语。

#### 1.1 项目结构

首先，请创建以下文件和目录结构：



#### 1.2 编写 `config.py` (安全配置)

我们将在此文件中定义用于认证的 API 密钥。**请注意，在生产环境中，这些密钥应通过环境变量或更安全的秘密管理系统加载，而非直接硬编码在代码中。**


In [ ]:
# config.py
import os

# 在生产环境中，请通过环境变量设置这些密钥
# 例如：export MCP_API_KEY_HELLO_SERVER="your_super_secret_key_123"
API_KEYS = {
    "hello_server_key": os.getenv("MCP_API_KEY_HELLO_SERVER", "your_super_secret_key_123")
}

# 确保在运行前设置环境变量
# 例如：在Linux/macOS终端运行：
# export MCP_API_KEY_HELLO_SERVER="my_dev_key_for_hello"
# 在Windows PowerShell运行：
# $env:MCP_API_KEY_HELLO_SERVER="my_dev_key_for_hello"

重要提示： 请替换 your_super_secret_key_123 为一个你自己的强密码。在实际部署前，强烈建议将 MCP_API_KEY_HELLO_SERVER 设置为环境变量，而不是直接写入代码。

#### 1.3 编写 `requirements.txt`

我们的 MCP 服务端将使用 **FastAPI** 来构建 API，并使用 **Uvicorn** 作为 ASGI 服务器来运行它。此外，`python-dotenv` 包可以帮助你在开发环境中从 `.env` 文件加载环境变量，方便管理 API 密钥。

请在 `mcp_custom_server/` 目录下创建 `requirements.txt` 文件，并添加以下内容：

#### 1.4 编写 `server.py` (MCP 服务端逻辑)

现在，我们将编写 `server.py` 文件，它包含了一个 FastAPI 应用程序，用于实现一个受 API 密钥保护的 MCP 端点。这个端点将提供我们之前提到的“问候”功能。


In [ ]:
# server.py
from fastapi import FastAPI, Header, HTTPException, Depends
from fastapi.security import APIKeyHeader
from pydantic import BaseModel
import uvicorn
from config import API_KEY
from typing import List

app = FastAPI(
    title="Calculator MCP Server",
    description="一个简单的带安全认证的计算器 MCP 服务端",
    version="1.0.0"
)

# API key security scheme
api_key_header = APIKeyHeader(name="X-Api-Key")

async def verify_api_key(api_key: str = Depends(api_key_header)):
    """验证 API 密钥"""
    if api_key != API_KEY:
        raise HTTPException(
            status_code=401,
            detail="Invalid API key"
        )
    return api_key

class Numbers(BaseModel):
    """数字输入模型"""
    num1: float
    num2: float

class Result(BaseModel):
    """计算结果模型"""
    result: float

@app.post(
    "/add",
    response_model=Result,
    summary="Addition Service",
    description="计算两个数的和",
    operation_id="add_numbers"
)
async def add_numbers(
    numbers: Numbers,
    api_key: str = Depends(verify_api_key)
) -> Result:
    """计算两个数的和"""
    return Result(result=numbers.num1 + numbers.num2)

@app.get(
    "/capabilities",
    summary="获取服务能力",
    operation_id="get_capabilities"
)
async def get_capabilities():
    """返回服务能力描述"""
    return {
        "add": {
            "name": "Addition Service",
            "description": "计算两个数的和",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {
                        "type": "number",
                        "description": "第一个数"
                    },
                    "num2": {
                        "type": "number",
                        "description": "第二个数"
                    }
                },
                "required": ["num1", "num2"]
            },
            "security": {
                "api_key": {
                    "type": "apiKey",
                    "in": "header",
                    "name": "X-Api-Key"
                }
            }
        }
    }

@app.get(
    "/",
    summary="API Root",
    description="Returns basic information about the API",
    operation_id="get_root"
)
async def get_root():
    """返回 API 基本信息"""
    return {
        "name": app.title,
        "description": app.description,
        "version": app.version,
        "endpoints": [
            "/",
            "/add",
            "/capabilities"
        ],
        "mcp": {
            "version": "0.1.0",
            "capabilities": [
                "add"
            ]
        }
    }

@app.post(
    "/",
    summary="Root POST Handler",
    description="Handles POST requests to root path",
    operation_id="post_root"
)
async def post_root():
    """处理根路径的 POST 请求"""
    return {
        "status": "success",
        "message": "POST request received",
        "mcp": {
            "version": "0.1.0",
            "capabilities": [
                "add"
            ]
        }
    }

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


#### 1.5 安装依赖

在 `mcp_custom_server` 目录下，打开终端并运行以下命令，安装所有必要的 Python 库：

```bash
pip install -r requirements.txt

#### 1.6 运行自定义 MCP 服务端

在 `mcp_custom_server` 目录下，运行你的 Python 服务端：
```bash
python server.py

你将看到服务在 `http://127.0.0.1:8000` 上启动。**请保持这个终端窗口处于运行状态**，因为它是提供自定义 MCP 服务的基础。


### 2. 配置 Cursor 的 `mcp.json`

现在，我们需要将你刚刚创建的自定义 MCP 服务端添加到 Cursor IDE 所使用的 `mcp.json` 配置中。这将允许你的 AI Agent 识别并调用这个新的服务。同时，你需要在此处添加必要的**安全认证信息**，以便 MCP Client 知道如何向你的服务端发送 API 密钥。

在你的 `mcp.json` 文件中，添加一个新的服务器配置，如下所示。请注意 `security.api_key` 部分，它明确指示了 MCP Client 如何传递 API 密钥。

```json
{
  "mcpServers": {
    "bingcn": {
      "command": "npx",
      "args": [
        "bing-cn-mcp"
      ]
    },
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/Users/wilson/Desktop"
      ],
      "description": "Local filesystem access"
    },
    "custom_calculator_server": {
      "url": "http://127.0.0.1:8000",
      "description": "一个简单的计算器服务，用于计算两个数的和。需要通过 X-Api-Key 进行认证。",
      "security": {
        "api_key": {
          "header": "X-Api-Key",
          "value": "your_super_secret_key_123"
        }
      }
    }
  }
}
```

http://127.0.0.1:8000/capabilities

```json
{
  "add": {
    "name": "Addition Service",
    "description": "计算两个数的和",
    "parameters": {
      "type": "object",
      "properties": {
        "num1": {
          "type": "number",
          "description": "第一个数"
        },
        "num2": {
          "type": "number",
          "description": "第二个数"
        }
      },
      "required": [
        "num1",
        "num2"
      ]
    },
    "security": {
      "api_key": {
        "type": "apiKey",
        "in": "header",
        "name": "X-Api-Key"
      }
    }
  }
}
```

现在，在 Agent 的终端中尝试以下输入：

使用 mcp 的计算服务， 计算  12312.123 与  12312321.1231 的和


**预期输出**（经过优化和简化，模拟 Agent 的最终响应）：

计算结果已经出来了！12312.123 + 12312321.1231 = 12324633.2461。
这是通过 MCP 的计算服务精确得出的结果。还需要我帮你计算其他数字吗？

### 3. 生产级工具链的考量

将自定义 MCP 服务端与 Cursor (或其他 MCP Hosts) 结合，为构建**生产级 AI 工具链**奠定了基础。在实际应用中，你还需要考虑以下关键点：

* **更强大的认证机制：** 除了简单的 API Key，应采用 **OAuth2** 或 **JWT** 等更安全的认证流程。FastAPI 对此有良好支持。
* **日志与监控：** 为 MCP 服务端和 Agent 增加详细日志，并利用 **Prometheus、Grafana** 等工具进行实时监控，以便快速诊断问题。
* **错误处理与重试：** 实现鲁棒的错误处理和**自动重试机制**，应对网络故障、服务不可用等问题，提高系统韧性。
* **异步操作：** 对于耗时操作，采用**异步编程 (`asyncio`)** 避免阻塞，提升并发处理能力。
* **版本控制：** 明确 MCP Server 及其工具的能力版本，确保 Agent 调用兼容的接口。
* **部署策略：** 将 MCP Servers 作为**独立微服务**，通过 **Docker** 和 **Kubernetes** 等技术进行容器化管理和弹性扩展。
* **工具发现：** 生产级 Agent 通常需要更智能的工具发现机制，如从中心注册表动态加载工具描述。

---

### 小结

通过本任务，你已掌握如何从零构建**带安全认证的 MCP 服务端**，并将其集成到 Agent 的工具调用流程中。这为你将任何自定义逻辑转化为 AI 可用的工具，从而构建**复杂、安全且高效的 AI Agent** 奠定了坚实基础。

---